### Import libraries and Take a view of datasets

In [71]:
import pandas as pd
import numpy as np

In [72]:
patient_df = pd.read_csv('./raw/patients.csv')
patient_df.head()

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
0,3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
1,4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
2,6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
3,9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
4,11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [73]:
# Create total_los column = (dischtime - admittime)/(seconds in a day)
patient_df['admittime'] = pd.to_datetime(patient_df['admittime'])
patient_df['dischtime'] = pd.to_datetime(patient_df['dischtime'])

patient_df['total_los'] = ((patient_df['dischtime'] - patient_df['admittime']).dt.total_seconds() / (24 * 60 * 60)).round(4)

In [74]:
lab_df = pd.read_csv('./raw/vitals_labs_mean.csv')
lab_df.head()

,subject_id,hadm_id,icustay_id,hours_in,alanine aminotransferase,albumin,albumin ascites,albumin pleural,albumin urine,alkaline phosphate,...,total protein,total protein urine,troponin-i,troponin-t,venous pvo2,weight,white blood cell count,white blood cell count urine,ph,ph urine
0,3,145834,211552,0,25.0,1.8,NaN,NaN,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,14.842857,NaN,7.40,5.0
1,3,145834,211552,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.26,NaN
3,3,145834,211552,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,145834,211552,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
dianoses_df = pd.read_csv('./raw/DIAGNOSES_ICD.csv')
dianoses_df.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,4,3,145834,1.0,0389
4,5,3,145834,2.0,78559


In [76]:
print(patient_df.shape)
print(lab_df.shape)
print(dianoses_df.shape)

(34472, 32)
(2200954, 108)
(225345, 5)


In [77]:
print(patient_df['subject_id'].nunique())
print(lab_df['subject_id'].nunique())
print(dianoses_df['subject_id'].nunique())

34472
34472
23692


### Data Cleaning

Đầu tiên, chúng tôi phát hiện trong dữ liệu có nhiều bệnh nhân có thời gian bắt đầu điều trị ICU lần đầu tiên diễn ra trước thời gian nhập viện. 

Điều này có thể lí giải là nhiều bệnh nhân nhập viện trong tình trạng khẩn cấp (bị bệnh nặng, tai nạn hoặc thai phụ sinh sản) nên được đưa vào ICU trước khi làm thủ tục nhập viện. 

Chúng tôi giải quyết những trường hợp này bằng cách thay thời gian nhập viện bởi thời gian vào ICU lần đầu tiên.

(Trang 24, 25)

In [78]:
tmp_features = ['subject_id', 'hadm_id', 'icustay_id', 'admittime', 'intime']

# Check if admittime is always before intime
tmp = patient_df[patient_df['admittime'] > patient_df['intime']][tmp_features][:10]
tmp

,subject_id,hadm_id,icustay_id,admittime,intime
23,35,166707,282039,2122-02-10 11:15:00,2122-02-10 09:39:59
230,347,119310,268893,2118-05-23 13:30:00,2118-05-23 10:56:34
251,373,104540,211665,2198-08-27 12:15:00,2198-08-27 09:17:10
274,409,105471,224525,2159-09-17 12:00:00,2159-09-17 10:33:00
321,487,160958,228805,2130-10-31 12:15:00,2130-10-31 11:04:31
514,796,127253,253353,2137-11-11 12:45:00,2137-11-11 10:23:13
524,810,168366,257899,2136-01-18 21:13:00,2136-01-18 01:10:26
615,957,145966,292666,2162-03-22 11:45:00,2162-03-22 10:42:04
712,1108,108891,208969,2198-04-16 16:00:00,2198-04-16 13:06:00
912,1417,105034,236094,2129-06-17 23:30:00,2129-06-17 22:48:53


In [79]:
# Replace admittime with intime if admittime > intime
patient_df['admittime'] = np.where(patient_df['admittime'] > patient_df['intime'], patient_df['intime'], patient_df['admittime'])

In [80]:
tmp.to_csv("./latex_table/4.5.csv", index=False)

Chúng tôi còn quan sát từ dữ liệu có một số bệnh nhân có thời gian nhập viện xảy ra sau thời gian xuất viện. Điều này là vô lý, do đó chúng tôi loại bỏ những bệnh nhân này khỏi bộ dữ liệu.

(Trang 25)

In [81]:
# Admittime > Dischtime
tmp_features = ['subject_id', 'hadm_id', 'icustay_id', 'admittime', 'dischtime']

# Ensure dtypes are correct
patient_df['admittime'] = pd.to_datetime(patient_df['admittime'])
patient_df['dischtime'] = pd.to_datetime(patient_df['dischtime'])

tmp = patient_df[patient_df['admittime'] > patient_df['dischtime']][tmp_features]

In [82]:
len(tmp)

17

In [83]:
# Drop from patient_df
patient_df = patient_df[~patient_df['subject_id'].isin(tmp['subject_id'])]

In [84]:
tmp = tmp[:15]
tmp

,subject_id,hadm_id,icustay_id,admittime,dischtime
120,181,102631,246694,2153-10-12 09:49:00,2153-10-12 06:29:00
3376,5567,182804,233453,2181-07-09 15:31:00,2181-07-09 12:00:00
4147,6832,164068,208677,2142-05-23 13:20:00,2142-05-23 12:00:00
5186,8604,102784,218796,2110-01-06 15:15:00,2110-01-06 12:00:00
7232,11983,129454,207034,2138-08-14 14:12:00,2138-08-14 12:00:00
7774,12844,181672,285687,2192-05-07 12:24:00,2192-05-07 12:00:00
11215,18567,143379,202436,2133-02-08 15:24:00,2133-02-08 07:17:00
11559,19124,156524,275239,2139-10-12 04:08:00,2139-10-12 01:13:00
12272,20349,131432,278656,2183-12-10 16:49:00,2183-12-10 07:48:00
12824,21272,156385,291938,2162-09-25 07:36:00,2162-09-25 01:23:00


In [85]:
tmp.to_csv("./latex_table/4.6.csv", index=False)

Tiếp theo, chúng tôi phát hiện có 373 bệnh nhân có tổng thời gian nằm viện nhỏ hơn tổng thời gian nằm ICU. 

In [86]:
# Check los_icu > total_los + 0.5
tmp_features = ['subject_id', 'hadm_id', 'icustay_id', 'los_icu', 'total_los']
tmp = patient_df[patient_df['los_icu'] > patient_df['total_los'] + 0.5][tmp_features]
tmp

,subject_id,hadm_id,icustay_id,los_icu,total_los
279,417,178013,207928,1.341331,0.3722
427,657,165342,243768,3.455949,2.8062
569,886,130937,294616,2.261806,1.2625
585,908,134650,229578,2.276539,1.6250
602,937,148592,228181,5.253021,3.5562
...,...,...,...,...,...
33516,95991,173297,207431,7.487604,3.7083
33915,97609,166990,243152,2.443762,0.9812
34194,98797,105447,244147,1.238171,0.6625
34235,98995,123142,223471,1.945012,0.3944


In [87]:
tmp = tmp[:17]
tmp.to_csv("./latex_table/4.7.csv", index=False)

Bước cuối cùng trong phần này, chúng tôi tiến hành loại bỏ những bệnh nhân nằm viện quá 90 ngày. Lí do chúng tôi thực hiện điều này vì số lượng bệnh nhân dạng này rất ít (55 bệnh nhân).
Những bệnh nhân này có thể xem là trường hợp ngoại lệ

(trang 26)

In [88]:
# Drop total_los > 90
patient_df = patient_df[patient_df['total_los'] <= 90]

húng tôi dựa trên các nghiên cứu trước đó cho bài toán dự đoán số ngày nằm
viện, đặc biệt là các nghiên cứu trên bộ dữ liệu MIMIC-III, các tác giả thường đề xuất sử dụng hai nhóm đặc trưng sau đây:
+ Nhóm đặc trưng các chỉ số xét nghiệm của bệnh nhân.
+ Nhóm đặc trưng các thông tin cơ bản của bệnh nhâ

In [89]:
# Convert patient more than 90 ys to 300
patient_df['age'] = patient_df['age'].apply(lambda x: 300 if x > 90 else x)

In [90]:
patient_df.head()

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,...,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours,total_los
0,3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,...,6.064560,EMERGENCY,MICU,0,0,0,1,0,145,10.7847
1,4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,...,1.678472,EMERGENCY,MICU,0,0,0,1,0,40,7.7590
2,6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,...,3.672917,ELECTIVE,SICU,0,0,0,1,0,88,16.3646
3,9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,...,5.323056,EMERGENCY,MICU,1,1,1,1,0,127,4.8812
4,11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,...,1.584410,EMERGENCY,SICU,0,0,0,1,0,38,25.5292


In [91]:
col_to_keep = ['subject_id', 'hadm_id', 'icustay_id', 'gender', 'ethnicity', 'age', 'insurance', 'admittime', 'dischtime', 'intime', 'outtime', 'los_icu', 'admission_type', 'first_careunit',  'total_los']

In [92]:
patient_df = patient_df[col_to_keep]
patient_df.head()

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,insurance,admittime,dischtime,intime,outtime,los_icu,admission_type,first_careunit,total_los
0,3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 19:10:11,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,10.7847
1,4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,2191-03-23 18:41:00,2191-03-16 00:29:31,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,7.7590
2,6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,2175-06-15 16:00:00,2175-05-30 21:30:54,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,16.3646
3,9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 13:07:02,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,4.8812
4,11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,2178-05-11 19:00:00,2178-04-16 06:19:32,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,25.5292


In [93]:
# Dummy variables
patient_df = pd.get_dummies(patient_df, 
                            columns=['gender', 'ethnicity', 'first_careunit', 'insurance', 'admission_type'],
                            drop_first=True,
                            dtype='uint8')

patient_df.shape

(34418, 61)

In [94]:
# Drop admittime, dischtime, intime, outtime
patient_df.drop(columns=['admittime', 'dischtime', 'intime', 'outtime'], inplace=True)

In [95]:
patient_df.shape

(34418, 57)

In [96]:
patient_df.head()

,subject_id,hadm_id,icustay_id,age,los_icu,total_los,gender_M,ethnicity_AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE,ethnicity_ASIAN,ethnicity_ASIAN - ASIAN INDIAN,...,first_careunit_CSRU,first_careunit_MICU,first_careunit_SICU,first_careunit_TSICU,insurance_Medicaid,insurance_Medicare,insurance_Private,insurance_Self Pay,admission_type_EMERGENCY,admission_type_URGENT
0,3,145834,211552,76.526792,6.064560,10.7847,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
1,4,185777,294638,47.845047,1.678472,7.7590,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,6,107064,228232,65.942297,3.672917,16.3646,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,9,150750,220597,41.790228,5.323056,4.8812,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0
4,11,194540,229441,50.148295,1.584410,25.5292,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0


Bay gio su ly voi bang Diagnoses

In [97]:
dianoses_df[dianoses_df['icd9_code'].isnull()]

,row_id,subject_id,hadm_id,seq_num,icd9_code
86698,126401,11245,148742,NaN,NaN
96418,142003,12607,190435,NaN,NaN
132017,192550,17269,176570,NaN,NaN
136265,198343,17796,142890,NaN,NaN
178085,260938,23407,119146,NaN,NaN
182195,268004,23987,186175,NaN,NaN


In [98]:
dianoses_df.dropna(subset=['icd9_code'], inplace=True)

In [99]:
dianoses_df[:13].to_csv('./latex_table/4.8.csv', index=False)
dianoses_df[:13]

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,4,3,145834,1.0,0389
4,5,3,145834,2.0,78559
5,6,3,145834,3.0,5849
6,7,3,145834,4.0,4275
7,8,3,145834,5.0,41071
8,9,3,145834,6.0,4280
9,10,3,145834,7.0,6826


(Trang 28-29)
1. Đầu tiên, với mỗi mã bệnh trong cột Code ICD-9 trong Bảng 4.8, chúng tôi chỉ giữ lại 3
chữ số đầu tiên, lí do cho việc làm này của chúng tôi là các nhóm nhỏ của cùng một bệnh
có khả năng tương đồng vè mặt y tế cao.

In [100]:
dianoses_df.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,4,3,145834,1.0,0389
4,5,3,145834,2.0,78559


In [101]:
dianoses_df['icd9_code'] = dianoses_df['icd9_code'].astype(str)

In [102]:
# Take only the first 3 digits of the ICD9 code
dianoses_df['Recode'] = dianoses_df['icd9_code'].apply(lambda x: x[:3] if len(x) > 3 else x)


In [103]:
dianoses_df[:13].to_csv('./latex_table/4.9.csv', index=False)
dianoses_df[:13]

,row_id,subject_id,hadm_id,seq_num,icd9_code,Recode
0,1,2,163353,1.0,V3001,V30
1,2,2,163353,2.0,V053,V05
2,3,2,163353,3.0,V290,V29
3,4,3,145834,1.0,0389,038
4,5,3,145834,2.0,78559,785
5,6,3,145834,3.0,5849,584
6,7,3,145834,4.0,4275,427
7,8,3,145834,5.0,41071,410
8,9,3,145834,6.0,4280,428
9,10,3,145834,7.0,6826,682


2. Để thu hẹp miền giá trị các chẩn đoán hơn nữa, chúng tôi dựa vào danh sách 18 nhóm các
mã bệnh ở trên. Với mỗi mã bệnh sau khi thực hiện mã hóa ở bước số một sẽ được mã hóa
tiếp một lần nữa bằng cách gán nhãn nhóm bệnh tương ứng với mỗi mã bệnh.
+ Infectious (mã 001-139): Đây là các mã bệnh có liên quan đến bệnh truyền nhiễm và ký
sinh trùng,
+ Neoplasms (mã 140-239): Đây là mã bệnh có liên quan đến bệnh ung thư,
+ Endocrine (mã 240-279): Đây là mã các bệnh liên quan đến nội tiết, dinh dưỡng, chuyển
hóa và rối loạn miễn dịch,
+ Blood (mã 280-289): Đây là mã các bệnh về máu,
+ Mental (mã 290-319): Đây là mã các bệnh liên quan đến chứng rối loạn tâm thần,
+ Nervous (mã 320-389): Đây là mã các bệnh liên quan đến hệ thần kinh và các giác quan,
+ Circulatory (mã 390-459): Đây là các mã bệnh liên quan đến hệ tuần hoàn,
+ Respiratory (mã 460-519): Đây là mã các bệnh liên quan đến hệ hô hấp,
+ Digestive (mã 520-579): Đây là mã các bệnh có liên quan đến hệ tiêu hóa,
+ Genitourinary (mã 580-629): Đây là mã các bệnh có liên quan đến hệ sinh dục,
+ Pregnancy (mã 630-679): Đây là mã các bệnh có liên quan đến các biến chứng trong thời
gian mang thai, sinh nở và hậu sản,
+ Skin (mã 680-709): Đây là mã các bệnh có liên quan đến da,
+ Muscular (mã 710-739): Đây là mã các bệnh có liên quan đến hệ thống cơ xương,
+ Congenital (mã 740-759): Đây là mã các bệnh có liên quan đến các dị tật bẩm sinh,
+ Prenatal (mã 760-779): Đây là mã các bệnh có liên quan đến thời kỳ chu sinh,
+ Misc (mã 780-799): Đây là mã các bệnh liên quan đến các triệu chứng, dấu hiệu và tình
trạng chưa xác định rõ.
+ Injury (mã 800-999): Đây là mã các bệnh liên quan đến các chấn thương và bệnh ngộ
độc,
+ Misc (danh sách ICD-9 mã E và mã V): Đây là danh sách các bệnh liên quan đên các
tác động bên ngoài gây thương tích và một số phân loại bổ sung.

In [104]:
# coUNT distinct subject_id
dianoses_df['subject_id'].nunique()

23691

In [105]:
# Take look at Recode = 018
dianoses_df[dianoses_df['Recode'] == '018']

,row_id,subject_id,hadm_id,seq_num,icd9_code,Recode
38898,55917,5018,114327,3.0,01894,018
38903,55922,5018,154356,1.0,01894,018
45488,66057,5872,131850,2.0,01803,018
119574,174800,15716,107629,5.0,01894,018
119622,174848,15716,131088,6.0,01895,018
119642,174868,15716,158787,1.0,01805,018


[55917, 55922, 66057, 174800, 174848, 174868]

In [106]:
dianoses_df['Recode'] = np.where(
    
    # Recode = 18 if icd9_code starts with E or V
    dianoses_df['icd9_code'].str.startswith(('E', 'V')),
    18,  
    
    # Keep the first 3 digits of icd9_code
    dianoses_df['icd9_code'].str[:3].apply(pd.to_numeric, errors='coerce')
)

In [107]:
# Bins and labels for pd.cut
bins = [0, 139, 239, 279, 289, 319, 389, 459, 519, 579, 629, 679, 709, 739, 759, 779, 799, 999]
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

dianoses_df['Recode2'] = np.where(
    
    # Cut into bins
    dianoses_df['Recode'] != 18,
    pd.cut(dianoses_df['Recode'], bins=bins, labels=labels, right=True).astype(int),
    # Keep the value of Recode if Recode = 18
    18 
)

In [108]:
dianoses_df.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code,Recode,Recode2
0,1,2,163353,1.0,V3001,18.0,18
1,2,2,163353,2.0,V053,18.0,18
2,3,2,163353,3.0,V290,18.0,18
3,4,3,145834,1.0,0389,38.0,1
4,5,3,145834,2.0,78559,785.0,16


In [109]:
category_mapping = {
    1 : 'Infectious',
    2 : 'Neoplasms',
    3 : 'Endocrine',
    4 : 'Blood',
    5 : 'Mental',
    6 : 'Nervous',
    7 : 'Circulatory',
    8 : 'Respiratory',
    9 : 'Digestive',
    10 : 'Genitourinary',
    11 : 'Pregrancy',
    12 : 'Skin',
    13 : 'Muscular',
    14 : 'Congenital',
    15 : 'Perinatal',
    16 : 'Misc',
    17 : 'Injury',
    18 : 'Misc (code E or V)'
}

In [110]:
dianoses_df['Category'] = dianoses_df['Recode2'].map(category_mapping)

In [111]:
dianoses_df.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code,Recode,Recode2,Category
0,1,2,163353,1.0,V3001,18.0,18,Misc (code E or V)
1,2,2,163353,2.0,V053,18.0,18,Misc (code E or V)
2,3,2,163353,3.0,V290,18.0,18,Misc (code E or V)
3,4,3,145834,1.0,0389,38.0,1,Infectious
4,5,3,145834,2.0,78559,785.0,16,Misc


In [112]:
# Now fix row_id 55917 and 55922 with Recode2 = 1 and Category = 'Infectious'
dianoses_df.loc[dianoses_df['row_id'].isin([55917, 55922, 66057, 174800, 174848, 174868]), 'Recode2'] = 1
dianoses_df.loc[dianoses_df['row_id'].isin([55917, 55922, 66057, 174800, 174848, 174868]), 'Category'] = 'Infectious'

In [113]:
# Remove Recode, rename Recode2 to Recode
dianoses_df.drop(columns=['Recode'], inplace=True)
dianoses_df.rename(columns={'Recode2': 'Recode'}, inplace=True)

In [114]:
dianoses_df[:13].to_csv('./latex_table/4.10.csv', index=False)
dianoses_df[:13]

,row_id,subject_id,hadm_id,seq_num,icd9_code,Recode,Category
0,1,2,163353,1.0,V3001,18,Misc (code E or V)
1,2,2,163353,2.0,V053,18,Misc (code E or V)
2,3,2,163353,3.0,V290,18,Misc (code E or V)
3,4,3,145834,1.0,0389,1,Infectious
4,5,3,145834,2.0,78559,16,Misc
5,6,3,145834,3.0,5849,10,Genitourinary
6,7,3,145834,4.0,4275,7,Circulatory
7,8,3,145834,5.0,41071,7,Circulatory
8,9,3,145834,6.0,4280,7,Circulatory
9,10,3,145834,7.0,6826,12,Skin


Bay giờ, tạo thêm 1 cột amount để đếm số  Category trùng nhau ứng với mỗi bệnh nhân.

Ví dụ: bênh nhân 3 thì các hàng Circulatory có cột amount là 4

In [115]:
tmp = pd.get_dummies(dianoses_df, columns=['Category'], dtype='uint8')
tmp.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code,Recode,Category_Blood,Category_Circulatory,Category_Congenital,Category_Digestive,...,Category_Mental,Category_Misc,Category_Misc (code E or V),Category_Muscular,Category_Neoplasms,Category_Nervous,Category_Perinatal,Category_Pregrancy,Category_Respiratory,Category_Skin
0,1,2,163353,1.0,V3001,18,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2,2,163353,2.0,V053,18,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3,2,163353,3.0,V290,18,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,4,3,145834,1.0,0389,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,3,145834,2.0,78559,16,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [116]:
# Groupby theo subject_id và sum các dummy variables
grouped = tmp.groupby('subject_id').agg({
    col: 'sum' for col in tmp.columns if col not in ['subject_id', 'row_id', 'hadm_id', 'seq_num', 'icd9_code', 'Recode']
}).reset_index()

Bay gio ta thay the ten cac ' ' thanh 
'_' trong ten cac cot trong `vital_lab` (trang 39)

In [117]:
# Drop from lab_df
lab_df = lab_df[lab_df['subject_id'].isin(patient_df['subject_id'])]

In [118]:
lab_names = list(lab_df.columns[4:])
lab_names = [c.replace(' ', '_') for c in lab_names]
lab_df.columns = list(lab_df.columns[:4]) + lab_names
print(len(lab_names))

104


In [119]:
lab_names_first_last = []
for name in lab_names:
    lab_names_first_last.append(name + '_first')
    lab_names_first_last.append(name + '_last')

In [120]:
# Take first 24h each patient
lab_first_24h = lab_df[lab_df['hours_in'] <= 24]

# Take min and max of each hour_in for each patient
lab_first_24h = lab_first_24h.groupby('subject_id').agg({
    col: ['min', 'max'] for col in lab_names
}).reset_index()

In [121]:
# Flatten the columns
lab_first_24h.columns = ['subject_id'] + lab_names_first_last

lab_first_24h.head()

,subject_id,alanine_aminotransferase_first,alanine_aminotransferase_last,albumin_first,albumin_last,albumin_ascites_first,albumin_ascites_last,albumin_pleural_first,albumin_pleural_last,albumin_urine_first,...,weight_first,weight_last,white_blood_cell_count_first,white_blood_cell_count_last,white_blood_cell_count_urine_first,white_blood_cell_count_urine_last,ph_first,ph_last,ph_urine_first,ph_urine_last
0,3,25.0,25.0,1.8,1.8,NaN,NaN,NaN,NaN,NaN,...,106.000000,106.000000,14.842857,24.4,35.0,35.0,7.26,7.4075,5.0,5.0
1,4,24.0,24.0,2.8,2.8,NaN,NaN,NaN,NaN,NaN,...,53.599998,53.599998,9.700000,9.7,NaN,NaN,7.47,7.4700,7.0,7.0
2,6,23.0,23.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.600000,10.6,NaN,NaN,NaN,NaN,NaN,NaN
3,9,11.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.500000,13.7,NaN,NaN,7.39,7.4300,5.0,8.0
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.300000,12.8,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# Make sure that subject_id are the same in 3 tables
print(len(patient_df['subject_id'].unique()))
print(len(grouped['subject_id'].unique()))
print(len(lab_first_24h['subject_id'].unique()))

34418
23691
34418


In [132]:
# Print the number of subject_id in grouped but not in patient_df
patient_df_subject_id = set(patient_df['subject_id'])
grouped_subject_id = set(grouped['subject_id'])
lab_df_subject_id = set(lab_first_24h['subject_id'])

9999
9999
0


In [134]:
# Join 3 tables (INNER JOIN)
df = patient_df.merge(grouped, on='subject_id', how='inner')
df = df.merge(lab_first_24h, on='subject_id', how='inner')

df.head()

,subject_id,hadm_id,icustay_id,age,los_icu,total_los,gender_M,ethnicity_AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE,ethnicity_ASIAN,ethnicity_ASIAN - ASIAN INDIAN,...,weight_first,weight_last,white_blood_cell_count_first,white_blood_cell_count_last,white_blood_cell_count_urine_first,white_blood_cell_count_urine_last,ph_first,ph_last,ph_urine_first,ph_urine_last
0,3,145834,211552,76.526792,6.064560,10.7847,1,0,0,0,...,106.0,106.0,14.842857,24.4,35.0,35.0,7.26000,7.4075,5.0,5.0
1,9,150750,220597,41.790228,5.323056,4.8812,1,0,0,0,...,NaN,NaN,7.500000,13.7,NaN,NaN,7.39000,7.4300,5.0,8.0
2,11,194540,229441,50.148295,1.584410,25.5292,0,0,0,0,...,NaN,NaN,9.300000,12.8,NaN,NaN,NaN,NaN,NaN,NaN
3,12,112213,232669,72.374177,7.634815,12.6958,1,0,0,0,...,NaN,NaN,7.800000,8.4,NaN,NaN,7.08375,7.3500,NaN,NaN
4,13,143045,263738,39.866118,3.666042,6.8556,0,0,0,0,...,NaN,NaN,16.600000,19.3,NaN,NaN,7.34000,7.5100,6.5,6.5


In [136]:
# Move total_los to the last column
cols = list(df.columns)
cols.remove('total_los')
cols.append('total_los')

df = df[cols]

In [137]:
df.to_csv('./cleaned/data.csv', index=False)

In [139]:
df.shape

(13692, 283)

## 